In [2]:
import pygame
import sys
import random
import numpy as np
import time
import copy
from numpy.core.multiarray import ndarray
from pygame.math import Vector2
from typing import List

def relu(a):
    return a * (a > 0)


def softmax(x):
    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x), axis=1)

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class Snake:
    def __init__(self, display=False):
        self.body = [Vector2(1, 0), Vector2(0, 0)]
        self.dir = Vector2(1, 0)
        self.is_grow_up = False

        if display:
            self.hor_part = pygame.image.load('objects/hor_part.png').convert_alpha()
            self.vert_part = pygame.image.load('objects/vert_part.png').convert_alpha()

            self.down_right_part = pygame.image.load('objects/down_right_part.png').convert_alpha()
            self.down_left_part = pygame.image.load('objects/down_left_part.png').convert_alpha()
            self.up_right_part = pygame.image.load('objects/up_right_part.png').convert_alpha()
            self.up_left_part = pygame.image.load('objects/up_left_part.png').convert_alpha()

            self.head_up = pygame.image.load('objects/head_up.png').convert_alpha()
            self.head_down = pygame.image.load('objects/head_down.png').convert_alpha()
            self.head_right = pygame.image.load('objects/head_right.png').convert_alpha()
            self.head_left = pygame.image.load('objects/head_left.png').convert_alpha()

            self.tail_up = pygame.image.load('objects/tail_up.png').convert_alpha()
            self.tail_down = pygame.image.load('objects/tail_down.png').convert_alpha()
            self.tail_right = pygame.image.load('objects/tail_right.png').convert_alpha()
            self.tail_left = pygame.image.load('objects/tail_left.png').convert_alpha()

    def place_snake(self):
        self.pick_head_dir()
        self.pick_tail_dir()

        for idx, part in enumerate(self.body):
            x, y = part.x * BLOCK_SIZE, part.y * BLOCK_SIZE
            part_rect = pygame.Rect(x, y, BLOCK_SIZE, BLOCK_SIZE)

            if idx == 0:
                image = self.head

            elif idx + 1 == len(self.body):
                image = self.tail

            else:
                prev, next = self.body[idx + 1] - part, self.body[idx - 1] - part
                if prev.y == next.y:
                    image = self.hor_part
                elif prev.x == next.x:
                    image = self.vert_part
                else:
                    if prev.x == 1 and next.y == 1 or prev.y == 1 and next.x == 1:
                        image = self.down_right_part
                    elif prev.x == -1 and next.y == -1 or prev.y == -1 and next.x == -1:
                        image = self.up_left_part
                    elif prev.x == -1 and next.y == 1 or prev.y == 1 and next.x == -1:
                        image = self.down_left_part
                    elif prev.x == 1 and next.y == -1 or prev.y == -1 and next.x == 1:
                        image = self.up_right_part

            screen.blit(image, part_rect)

    def move_snake(self):
        head_pos = self.body[0] + self.dir

        if self.is_grow_up:
            self.body.insert(0, head_pos)
            self.is_grow_up = False
        else:
            body_new_pos = self.body[:-1]
            body_new_pos.insert(0, head_pos)
            self.body = body_new_pos[:]

    def grow(self):
        self.is_grow_up = True

    def pick_head_dir(self):
        head_dir = self.body[1] - self.body[0]

        if head_dir == Vector2(0, 1):
            self.head = self.head_up
        if head_dir == Vector2(0, -1):
            self.head = self.head_down
        if head_dir == Vector2(-1, 0):
            self.head = self.head_right
        if head_dir == Vector2(1, 0):
            self.head = self.head_left

    def pick_tail_dir(self):
        tail_dir = self.body[-2] - self.body[-1]

        if tail_dir == Vector2(0, 1):
            self.tail = self.tail_up
        if tail_dir == Vector2(0, -1):
            self.tail = self.tail_down
        if tail_dir == Vector2(-1, 0):
            self.tail = self.tail_right
        if tail_dir == Vector2(1, 0):
            self.tail = self.tail_left

In [4]:
class Food:
    def __init__(self):
        self.random_food_pos()

    def place_food(self):
        x, y = self.pos.x * BLOCK_SIZE, self.pos.y * BLOCK_SIZE
        food_rect = pygame.Rect(x, y, BLOCK_SIZE, BLOCK_SIZE)
        screen.blit(pear, food_rect)

    def random_food_pos(self):
        self.x = random.randint(0, BLOCK_SIZE - 1)
        self.y = random.randint(0, BLOCK_SIZE - 1)
        self.pos = Vector2(self.x, self.y)

In [5]:
class Game:
    def __init__(self, thetas1: ndarray, thetas2: ndarray, with_display = False):
        self.snake = Snake(with_display)
        self.food = Food()
        self.allowed_steps = 200 # How many steps without getting a food is ok

        self.thetas1 = thetas1
        self.thetas2 = thetas2

    def look_in_direction(self, direction: Vector2):
        '''
        The function that return a distances to food, wall and body of the snake in given direction
        :param direction: Vector of direction from pygame
        :return: 3 values corresponding to distances to food, wall, body
        '''
        food_counter, obstacle_counter, body_counter = 0, 0, 0

        head_copy = self.snake.body[0].copy()
        counter = 0
        while not (head_copy.x < 0 or head_copy.x >= BLOCK_SIZE or
                   head_copy.y < 0 or head_copy.y >= BLOCK_SIZE):
            counter += 1
            head_copy += direction
            for part in self.snake.body[1:]:
                if part == head_copy:
                    body_counter = counter
            if head_copy == self.food.pos:
                food_counter = counter
        obstacle_counter = counter
        return food_counter, obstacle_counter, body_counter

    def get_distances(self):  # returns distances to food and closest obstacle for 5 directions
        '''
        Generates an input for NN, considering 7 direction (all by rotating by 45 degrees except 180), for every direction getting distances to food, wall and body
        :return:
        '''
        dir = self.snake.dir
        a, b, c = self.look_in_direction(Vector2(dir.y, -dir.x)) #90d
        d, e, f = self.look_in_direction(dir) #0d
        g, h, i = self.look_in_direction(Vector2(-dir.y, dir.x)) #-90d

        dir45 = Vector2(dir.x-dir.y, dir.x+dir.y) #45d
        dirmin45 = Vector2(dir.x+dir.y, dir.y-dir.x) #-45d
        j,k,l = self.look_in_direction(dir45)
        m,n, o = self.look_in_direction(dirmin45)

        dir135 = Vector2(-dir.x-dir.y, dir.x-dir.y) #135d
        dirmin135 = Vector2(-dir.x+dir.y, -dir.y-dir.x) #-135d
        s,t,u = self.look_in_direction(dir135)
        v,w,s = self.look_in_direction(dirmin135)
        return np.array([a, b, c, d, e, f,g,h,i,j, k, l, m, n, o,s,t,u,v,w,s])

    def decision(self):
        '''
        Implementation of 1 layer NN with ReLU act. function in hidden layer, and softmax in output layer of 3 neurons saying whether snake should turn right or left or not
        :return: decision: -1, 1 or 0
        '''
        input = self.get_distances()
        input = np.append(input, 1)
        layer1 = input @ self.thetas1
        layer1 = relu(layer1)
        layer1 = np.append(layer1, 1)
        output = layer1 @ self.thetas2
        output = output.reshape([1, -1])
        output = softmax(output)
        return np.argmax(output) - 1

    def grid(self):
        '''
        Print a grid on the game panel
        :return: None
        '''
        for row in range(BLOCK_SIZE):
            for col in range(BLOCK_SIZE):
                if (row + col) % 2 == 0:
                    color = LIGHT_GREEN
                else:
                    color = GREEN

                pygame.draw.rect(screen, color, [col * BLOCK_SIZE,
                                                 row * BLOCK_SIZE,
                                                 BLOCK_SIZE,
                                                 BLOCK_SIZE]
                                 )

    def place_score(self):
        '''
        Places a score on pygame panel
        :return: None
        '''
        score = len(self.snake.body) - 2

        score_text = f'score: {score}'
        score_surface = font.render(score_text, True, BLACK)
        score_rect = score_surface.get_rect(topright=(BLOCK_SIZE ** 2 - 25,
                                                      BLOCK_SIZE ** 2 - 25)
                                            )
        screen.blit(score_surface, score_rect)

    def place_objects(self) -> None:
        '''
        Places food and snake on pygame panel
        :return: None
        '''
        self.food.place_food()
        self.snake.place_snake()

    def update(self) -> bool:
        '''
        Updates a game. Like a new frame
        :return: bool showing whether the game is over
        '''
        self.snake.move_snake()
        self.check_objects()
        if self.allowed_steps <= 0:
            return True
        if self.check_out_of_screen():
            return True
        if self.check_body_collision():
            return True
        return False

    def check_objects(self):
        '''
        Check for getting food
        :return: None
        '''
        snake_head = self.snake.body[0]
        if snake_head == self.food.pos:
            self.allowed_steps = 200
            self.food.random_food_pos()
            self.snake.grow()
        else:
            self.allowed_steps -= 1

    def check_out_of_screen(self):
        '''
        Check if snake is out of screen
        :return: bool
        '''
        snake_head = self.snake.body[0]
        if (snake_head.x < 0 or snake_head.x >= BLOCK_SIZE or
                snake_head.y < 0 or snake_head.y >= BLOCK_SIZE):
            return True
        return False

    def check_body_collision(self):
        '''
        Checks if snake is in its body
        :return: bool
        '''
        snake_head = self.snake.body[0]
        for part in self.snake.body[1:]:
            if part == snake_head:
                return True
        return False

In [6]:
BLOCK_SIZE = 20 #number of cells in row and column
# next is rgb codes
GREEN = (120, 177, 90)
LIGHT_GREEN = (133, 187, 101)
BLACK = (20, 67, 76)
WHITE = (255, 255, 255)

In [7]:
def play_game(genes, with_display=False):
    '''
    Plays a game and returns a score
    :param genes: configuration for NN
    :param with_display: show a pygame interface or not
    :return: score
    '''
    game = Game(genes[0], genes[1], with_display)

    while True:
        if with_display:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

        decision = game.decision()
        if decision == -1:
            dir = game.snake.dir
            game.snake.dir = Vector2(-dir.y, dir.x)
        elif decision == 1:
            dir = game.snake.dir
            game.snake.dir = Vector2(dir.y, -dir.x)

        if game.update():
            if with_display:
                pygame.quit()
            return len(game.snake.body) - 2
        if with_display:
            game.grid()
            game.place_objects()
            game.place_score()

            pygame.display.update()
            time.sleep(0.05)

In [8]:
def generate_random_genoms():
    return [np.random.randn(22, 15) * 0.6, np.random.randn(16, 3) * 0.6]


def generate_population() -> List[List[ndarray]]:
    return [generate_random_genoms() for _ in range(population_size)]


def calculate_population_fitnesses(population: List[List[ndarray]]):
    results = []
    for ind_genes in population:
        results.append(play_game(ind_genes, False))  # play a game, record the result
    return results


def selection(population: List[List[ndarray]], fitness: List, n: int) -> List[List[ndarray]]:
    '''
    Perform selection from given population. Returns top-n individuals
    :param population: population
    :param fitness: pre calculated fitness scores
    :param n: how many best individuals to return
    :return: new list of individuals
    '''
    indices = np.argsort(fitness)
    population = [population[i] for i in indices]
    return population[-n:]


def crossover_ind(parents: List[List[ndarray]]) -> List[ndarray]:
    '''
    Performs a crossover. Done by creating an offspring with random genes and then, picking a values to genes from either parent 1(prob=p) or parent2(prob = 1-p)
    :param parents: list of parents available to choose from
    :return: list of individuals
    '''
    parent1 = parents[np.random.randint(0, len(parents))]
    parent2 = parents[np.random.randint(0, len(parents))]
    children = generate_random_genoms()
    for index_of_table in range(len(children)):
        for row in range(children[index_of_table].shape[0]):
            for column in range(children[index_of_table].shape[1]):
                p = np.random.uniform(0, 1)
                if p < crossover_rate:
                    children[index_of_table][row][column] = parent2[index_of_table][row][column]
                else:
                    children[index_of_table][row][column] = parent1[index_of_table][row][column]
    return children


def crossover_pop(parents: List[List[ndarray]], n: int) -> List[List[ndarray]]:
    '''
    Performs n crossovers in population
    :param parents:
    :param n:
    :return:
    '''
    offsprings = []
    for i in range(n):
        offsprings.append(crossover_ind(parents))
    return offsprings


def mutate(offsprings: List[List[ndarray]], p: float) -> List[List[ndarray]]:
    '''
    Mutate given offsprings. Every gene gets added to some uniformly distributed value from 0.1 to 0.1 with probability p
    :param offsprings: list of offsprings
    :param p: probability of mutation of gene
    :return: mutated offsprings
    '''
    mutated_offsprings = []
    for offspring in offsprings:  # all offsprings
        cur_offspring = copy.deepcopy(offspring)
        for weight in cur_offspring:  # all lists
            for _ in range(int(weight.shape[0] * weight.shape[1] * p)):
                row = random.randint(0, weight.shape[0] - 1)
                col = random.randint(0, weight.shape[1] - 1)
                weight[row, col] += random.uniform(-0.15, 0.15)
        mutated_offsprings.append(cur_offspring)
    return mutated_offsprings

In [10]:
counter = 0
best_result = 20 # created for recording best result ever
best_fitness = 0 # best result in generation
population_size = 100  # how many snakes
mutation_probability = 0.3
generations = 200
wait_time_for_improve = 30
crossover_rate = 0.3  # probability that some weight of NN in parent 1 will be replaced by that of parent 2 in children
population = generate_population()
for gen_num in range(generations):
    fitness = calculate_population_fitnesses(population)
    if gen_num%10==0:
      print(fitness)
    best_fitness = np.max(fitness)
    if best_fitness > best_result:
      best_result = best_fitness
      best_answer = population[np.argmax(fitness)]
      counter = 0
    if counter >= wait_time_for_improve and best_fitness > 30:
      print("Early stopping")
      break
    counter+=1
    if gen_num%10==0:
      print("Generation {}, best fitness: {}".format(gen_num, best_fitness if best_fitness<20 else best_result))
    parents = selection(population, fitness, int(population_size / 2))
    offsprings = crossover_pop(parents, int(population_size / 2))
    offsprings = mutate(offsprings, mutation_probability)
    population = offsprings + parents  # new population is the combination of offsprings and parents

fitness = calculate_population_fitnesses(population)
best_fitness = np.max(fitness)
if best_fitness > best_result:
  best_result = best_fitness
  best_answer = population[np.argmax(fitness)]

print("Overall best fitness: {}".format(best_result))
answer = best_answer


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Generation 0, best fitness: 1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 3]
Generation 10, best fitness: 3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]
Generation 20, best fitness: 3
[0, 0

In [11]:
# Next is visualization of playing with best result from GA
screen = pygame.display.set_mode((BLOCK_SIZE ** 2, BLOCK_SIZE ** 2))
pear = pygame.image.load('objects/pear_fruit.png').convert_alpha()
pygame.font.init()
font = pygame.font.SysFont('umeminchos3', 20)
pygame.display.set_caption('Snake game')

play_game(answer, True)

32